In [18]:
#Question 2.1

%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt


In [ ]:
acd_df = pd.read_csv('Accidents0514.csv')
vec_df = pd.read_csv('Vehicles0514.csv')
cas_df = pd.read_csv('Casualties0514.csv')

In [3]:
#Number of accidents occured in urban areas
urban_count = acd_df['Urban_or_Rural_Area'][(acd_df['Urban_or_Rural_Area'] == 1)].value_counts()

# fraction of accidents occur in urban areas: 0.64265691087
print ('%.10f' % (urban_count[1]/len(acd_df)))

0.6426569109


In [4]:
#Question 2.2
acd_df['Hours'] = acd_df['Time'].str[:2] #Extract hours
hour_count = (acd_df['Hours'].value_counts()).sort_index() #Total accident occured at different hours
hour_count_fatal = (acd_df['Hours'][(acd_df['Accident_Severity'] == 3)].value_counts()).sort_index() #Fatal accident occured in different hours
danger = (hour_count_fatal / hour_count).sort_values(ascending=False) #corresponding frequency of fatal accidents to all accidents in different hour
print ('%.10f' % danger[1]) # the most dangerous time to drive is 8am with a fatal frequancy of 0.8808645681

0.8808645681


In [5]:
# Question 2.3

acd_df['Year2'] = acd_df['Date'].str[-4:] #Extract years
year_count = (acd_df['Year2'].value_counts()).sort_index() #Total accident in dif years
year_count = year_count.reset_index() 
year_count.columns = ['Year','Count']
#Find the slope
def trend(count, order=1):
    coeff = np.polyfit(year_count.index.values, list(count), order)
    slope = coeff[-2]
    return float(slope)

slope = trend(year_count['Count'])
print ('%.10f' %slope) #the slope in units of increased number of accidents per year

-6511.7272727273


In [6]:
#Question 2.4
spd_count = acd_df['Speed_limit'].value_counts().sort_index()
spd_count = spd_count.reset_index()
spd_count.columns = ['Speed','Count']
report = acd_df.groupby(['Speed_limit']).sum()
casualty = pd.Series(report['Number_of_Casualties'])
spd_count['Casualty'] = list(casualty) 
spd_count['ratio'] = spd_count['Casualty'] / spd_count['Count']
spd_count.corr()['Speed']['ratio']

0.96593905551148218

In [8]:
#Question 2.5
merged = pd.merge(vec_df,acd_df,left_on = 'Accident_Index', right_on = 'Accident_Index', sort = False)

#Skid and overturn report of all accidents when it's raining or snowing 
bad_weather = merged['Skidding_and_Overturning'][(merged['Weather_Conditions'] < 6 ) 
                                                 & (merged['Weather_Conditions'] > 1 )].value_counts().sort_index()

#Skid and overturn report of all accidents in nice weather with no high winds
good_weather = merged['Skidding_and_Overturning'][(merged['Weather_Conditions'] == 1)].value_counts().sort_index()

answer2_5 = (bad_weather[2:].sum()/bad_weather.sum())/(good_weather[2:].sum()/good_weather.sum())
print ('%.10f' %answer2_5) #Times difference is 1.7297218188

1.7297218188


In [9]:
#Question 2.6
sex = cas_df['Sex_of_Casualty'].value_counts().sort_index()
fatal = cas_df['Sex_of_Casualty'][(cas_df['Casualty_Severity'] == 1)].value_counts().sort_index()
male_fatal = fatal[1]/sex[1]
female_fatal = fatal[2]/sex[2]
answer2_6 = male_fatal/female_fatal
print ('%.10f' %answer2_6) #Times difference is 2.1317211281

2.1317211281


In [10]:
#Question 2.7
acd = acd_df.groupby(['Police_Force'])
long_std = acd['Longitude'].std()
lat_std = acd['Latitude'].std()

import math
trans = lat_std.apply(math.cos) 
max_area = (lat_std * 110.574) * (long_std * 111.320 * trans) * math.pi
max_area.max() # The largest district is 91 with area of 25167.124351883431 square kilometers

25167.124351883431

In [11]:
#Questions 2.8
age = cas_df['Age_of_Casualty'].value_counts().sort_index()    #pull age/accident data
age = age[19:]       #Filter our data below age 18

#Find the exponential fitting curve for the data
x = np.array(age.index.values, dtype=float)
y = np.array(age.values, dtype=float)
def model(x,a,b):
    return a*np.exp(-b*x) #b is the decay rate we are looking for based on definition

import scipy
curve = scipy.optimize.curve_fit(model, x, y, p0 = (1, 1e-5))
print (curve)
print ('%.10f' %curve[0][1]) #the decay rate 'b' is 0.0361883668

(array([  1.35945192e+05,   3.61883668e-02]), array([[  2.42916898e+07,   4.85493567e+00],
       [  4.85493567e+00,   1.14639027e-06]]))
0.0361883668
